In [13]:
import pandas as pd

%run Metric.ipynb
%run DataSource.ipynb
%run Account.ipynb


class Operation:
    
    # symbol=SZ#002335
    def __init__(self, account, symbol, short, mid):
        self.account = account
        self.symbol = symbol
        self.short = short
        self.mid = mid
        
        self.floating_profits = []
        self.operation_detail = pd.DataFrame(columns=['date','profit_rate','cash'])
        
        self.metric = Metric(self.symbol, level_day, self.short, self.mid)


    def get_fix_stop_loss_percent(self):
        return 7


    # 止损价格
    def get_stop_loss_price(self,price):
        return price * (1-get_fix_stop_loss_percent()/100)


    #     单笔交易承担亏损3%
    def buy_how_many(self, account_money, price, risk_percent=3):
        risk_money = account_money*risk_percent/100

    #     止损价格
        stop_loss_price = self.get_stop_loss_price(price)
        plan_shares = risk_money/(price-stop_loss_price)

        return int((plan_shares)/100)*100

    def sell_how_many(self):
        return self.account.get_shares()

    def open_opsition(self,date,price):
        account_cash = self.account.get_all_cash()
        
        buy_shares = self.buy_how_many(account_cash,price)
            
        print(date +' BUY price:'+ tostr(price) + ' shares:'+tostr(buy_shares)+ 
              ' space:'+tostr(buy_shares*price*100/self.account.get_all_cash()))

        self.account.buy_stock(date,price,buy_shares)
        
        
    def sell_all(self,date, price, sell_type='sell_point'):
        sell_shares = self.sell_how_many()
            
        self.account.sell_stock(date,price,sell_shares)
        
        profit_rate = (price-self.account.get_last_time_buy_price())*100/self.account.get_last_time_buy_price()

        print(date +' '+sell_type+' ' +tostr(sell_shares)+' shares, profit rate:'+tostr(profit_rate)+'%')
        print('floating_profits:'+str(self.floating_profits))
        print()
            
        self.operation_detail.loc[self.operation_detail.shape[0]+1] = [date,profit_rate,self.account.get_all_cash()]
        self.floating_profits.clear()
        
        
    def watching_for_sell(self,cur_date):
        future_part = self.metric.next_series(cur_date,10000)
        
        for index,row in future_part.iterrows():
            price = row['close']
            date = row['date']
            
            if(self.account.has_shares()>0):
                floating_profit_percent = (price-self.account.get_last_time_buy_price())*100/self.account.get_last_time_buy_price()
                self.floating_profits.append(round(floating_profit_percent,2))

                if(self.is_sell_point(date)):
                    self.sell_all(date,price,'sell_point')
                    return

#                 if(is_stop_loss(date,self.last_buy_date,self.last_buy_price)):
#                     self.sell_all(date,price,'stop_loss')

    
    def get_operations(self):
        return self.operation_detail
    
    def is_ma_cross_buy_point(self,cur_date):
        return self.metric.is_up_cross(cur_date)
    
    
    def is_ma_cross_sell_point(self,cur_date):
        return self.metric.is_down_cross(cur_date)
    
        
    def is_ma_cross_buy_point_delay_confirm(self,cur_date):
        latest_n=3
        up_crossed = self.latest_n_period_up_crossed(cur_date, latest_n)
        keep_up = self.k_on_ma(cur_date, self.short)

        if(up_crossed == True and keep_up == True):
            if(debug):
                print(cur_date+' up cross and keep 3 k lines up on MA('+ str(short)+')')
            return True
        else:
            return False
        
    def is_ma_cross_sell_point_delay_confirm(self, cur_date):
        down_cross = self.is_down_cross(cur_date)

        latest_n = 3
        latest_n_break_ma = self.latest_n_k_break_ma(cur_date, latest_n)

        deep_percent=3
        deep_break_ma = self.deep_k_break_ma(cur_date,deep_percent)

        if(down_cross == True):
            if(debug):
                print('down_cross')
            return True

        if(latest_n_break_ma == True):
            if(debug):
                print('latest_'+str(latest_n)+'_break_ma_'+str(mid))
            return True

        if(deep_break_ma == True):
            if(debug):
                print('deep_'+str(deep_percent)+'%_break_ma_'+str(mid))
            return True

        return False
    
        
    def is_buy_point(self,cur_date):
        return self.is_ma_cross_buy_point(cur_date)
        
        
    def is_sell_point(self,cur_date):
        return self.is_ma_cross_sell_point(cur_date)
    
    
    def is_stop_loss(self,cur_date,last_buy_date,last_buy_price):
        price = self.data[self.data['date']==cur_date]['close'].values[0]

        loss_percent = get_fix_stop_loss_percent()
        stop_loss_price=last_buy_price*(1-loss_percent/100)
        if(price<stop_loss_price):
            return True
        else:
            return False


    # max_profit_rate % 0.6
    def is_stop_profit_v1(self,floating_profits,taking=40):
        if(len(floating_profits)<1):
            return False

        max_profit = max(floating_profits)
        last_one = floating_profits[-1]

        taking_from_max = max_profit*(1-taking/100)
        if(max_profit>10 and last_one<taking_from_max):
            return True
        else:
            return False


    def is_stop_profit(self,floating_profits,taking=40):
        return self.is_stop_profit_v1(floating_profits)



    def send_go_signal(self,outburst, pre_amplitude, pre_ma_per, amplitude_threshold=60):
        if((outburst==True) and (pre_ma_per>-10)):
            return 'Buy'

        return 'Hold'
    
    def get_vol_features(self,cur_date):
        vol = self.get_vol(cur_date)
        
        vol_features = []
        for long in [10,20,40,60]:
            vol_features.append(int(vol/self.ma_vol(cur_date,long)))
            for short in [2,4,8,16]:
                temp_feature = int(self.ma_vol(cur_date,short)/self.ma_vol(cur_date,long))
                vol_features.append(temp_feature)

        return vol_features
    
    
    def get_score_by_year_period_profits(year_period_profits):
        win_period = 0

        for i in range(0,len(year_period_profits)):
    #         该年盈利百分比大于10%算为一个盈利年
            if(year_period_profits[i]>10):
                win_period += 1

    #         该年盈利百分比大于10%算为一个盈利年
            if(year_period_profits[i]<-10):
                win_period -= 1

        return int(win_period*100/len(year_period_profits))
            

    def get_score_by_max_retrace(retrace_percent):
        if(retrace_percent<=10):
            return 100-2*retrace_percent

        if(retrace_percent<20):
            return 80-2*(retrace_percent-10)

        if(retrace_percent<30):
            return 60-3*(retrace_percent-20)

        if(retrace_percent<40):
            return 30-3*(retrace_percent-30)

        if(retrace_percent<50):
            return 0-5*(retrace_percent-40)

        if(retrace_percent<60):
            return -50-5*(retrace_percent-50)

        return -100

